# 2-S5: A/B 테스트 기초

FDS에서 새 모델을 안전하게 검증하는 A/B 테스트를 학습합니다.

## 학습 목표
1. A/B 테스트란? (일반 vs FDS)
2. **Shadow 모드** (FDS 특수성)
3. **통계적 유의성 (p-value)** (면접 필수!)
4. Champion 승격 조건 4가지

## 예상 시간
- 약 1.5시간

## 선수 조건
- 2-S4 비용 최적화 + CI/CD 완료
- 기본 통계 개념

In [ ]:
# 필요한 라이브러리
import numpy as np
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료")

---
## 1. A/B 테스트란?

### 왜 A/B 테스트가 필요한가?

```
질문: 새 모델을 바로 배포하면 안 되나요?

문제점:
1. 테스트 데이터에서 좋았어도 실제 트래픽에선 다를 수 있음
2. 전체 사용자에게 한 번에 적용 -> 리스크 큼
3. 문제 발생 시 원인 파악 어려움

해결책: A/B 테스트
-> 일부 트래픽에만 새 모델 적용
-> 성능 비교 후 결정
```

### 일반 웹서비스 A/B 테스트

```
일반 웹서비스 (버튼 색상 테스트)

      사용자 요청
          |
    +-----+-----+
    |  50%    50%|
    v           v
 [버전 A]    [버전 B]
 파란 버튼   빨간 버튼
    |           |
    v           v
 응답 반환   응답 반환

-> 클릭률 비교: A=3.2%, B=4.1% -> B 채택!
```

### FDS A/B 테스트의 특수성

```
FDS에서 일반 A/B는 위험!

문제: 새 모델이 사기를 놓치면?
-> 실제 금전 피해 발생!
-> 웹 버튼 색상과 차원이 다른 리스크

해결: Shadow 모드 A/B 테스트
```

---
## 2. Shadow 모드 (FDS 핵심!)

### Shadow 모드 구조

```
Shadow 모드 A/B 테스트

      거래 요청
          |
    +-----+-----------+
    |                 |
    v                 v
 [Champion]       [Challenger]
 현재 모델         새 모델
    |                 |
    v                 v
 * 실제 응답 *      로깅만!
 (차단/승인)       (응답 안 함)

-> Challenger는 "그림자"처럼 따라다니며 예측만 기록
-> 실제 거래에 영향 없음 (리스크 제로)
-> 나중에 로그 분석으로 성능 비교
```

### Shadow 모드 장점

| 특성 | 일반 A/B | Shadow 모드 |
|------|---------|-------------|
| 실제 응답 | 두 모델 모두 | Champion만 |
| 리스크 | 있음 | **없음** |
| 트래픽 분할 | 필요 | 불필요 (100% 동시) |
| 비교 방식 | 실시간 메트릭 | 사후 로그 분석 |

In [ ]:
# 예제: Shadow 모드 구현 (의사 코드)

shadow_mode_code = """
# Shadow 모드 A/B 테스트 핵심 로직

class FDSService:
    def __init__(self):
        self.champion = load_model("models:/fds-model@champion")
        self.challenger = load_model("models:/fds-model@challenger")
    
    def predict(self, transaction):
        # 1. Champion으로 실제 예측 (응답에 사용)
        champion_pred = self.champion.predict(transaction)
        champion_proba = self.champion.predict_proba(transaction)
        
        # 2. Challenger는 Shadow 예측 (로깅만!)
        challenger_pred = self.challenger.predict(transaction)
        challenger_proba = self.challenger.predict_proba(transaction)
        
        # 3. 로깅 (나중에 비교 분석용)
        log_prediction(
            transaction_id=transaction.id,
            champion_pred=champion_pred,
            champion_proba=champion_proba,
            challenger_pred=challenger_pred,  # 그림자 예측
            challenger_proba=challenger_proba,
            actual_label=None  # 나중에 채워짐
        )
        
        # 4. Champion 결과만 반환 (Challenger는 응답 안 함!)
        return champion_pred, champion_proba
"""
print(shadow_mode_code)

In [ ]:
# 체크포인트 1: Shadow 모드 이해

shadow_mode_quiz = {
    "challenger_실제_응답": False,     # Challenger가 실제 응답을 반환하나요?
    "champion_실제_응답": True,        # Champion이 실제 응답을 반환하나요?
    "challenger_로깅_함": True,        # Challenger 예측을 로깅하나요?
    "실시간_비교": False,              # 실시간으로 성능을 비교하나요? (사후 분석)
    "리스크_제로": True,               # 실제 거래에 영향이 없나요?
}

# 검증
assert shadow_mode_quiz["challenger_실제_응답"] == False, "Challenger는 응답 안 함!"
assert shadow_mode_quiz["champion_실제_응답"] == True, "Champion만 응답!"
assert shadow_mode_quiz["challenger_로깅_함"] == True, "Challenger는 로깅만!"
assert shadow_mode_quiz["리스크_제로"] == True, "Shadow 모드는 리스크 없음!"

print("체크포인트 1 통과! Shadow 모드 이해 완료")
print("   -> Challenger는 '그림자'처럼 로깅만, 실제 응답은 Champion만!")

---
## 3. 통계적 유의성 (면접 필수!)

### 왜 통계 검정이 필요한가?

```
상황:

Champion Recall: 85.0%
Challenger Recall: 86.2%

"Challenger가 1.2%p 높으니까 바로 교체하면 되지 않나요?"

문제점:
- 그 차이가 실제 성능 차이인지?
- 아니면 우연히 발생한 차이인지?
- 내일 측정하면 반대로 나올 수도 있지 않나?

-> 통계적 검정으로 "우연이 아닌지" 확인해야 함!
```

### t-검정 (t-test) 기초

```
t-검정이란?

"두 그룹의 평균이 통계적으로 다른지" 검정하는 방법

예시:
- Champion 일별 Recall: [85, 84, 86, 85, 84, 85, 86]
- Challenger 일별 Recall: [87, 88, 87, 86, 88, 87, 88]

t-검정 결과:
- t-statistic: 두 그룹이 얼마나 다른지 (클수록 차이 큼)
- p-value: 그 차이가 우연일 확률
```

### p-value 해석

| p-value | 해석 | 결정 |
|---------|------|------|
| **< 0.01** | 매우 강한 증거 | 확실히 다름 |
| **< 0.05** | 강한 증거 | 다르다고 결론 (현업 기준) |
| **< 0.10** | 약한 증거 | 애매함, 더 데이터 필요 |
| **>= 0.10** | 증거 없음 | 차이 없다고 결론 |

> **현업 기준: p < 0.05** (5% 유의수준)

In [ ]:
# 예제: scipy로 t-검정 수행

# 두 모델의 일별 Recall (7일간 측정)
champion_recalls = np.array([0.85, 0.84, 0.86, 0.85, 0.84, 0.85, 0.86])
challenger_recalls = np.array([0.87, 0.88, 0.87, 0.86, 0.88, 0.87, 0.88])

print("성능 비교")
print(f"Champion 평균 Recall: {champion_recalls.mean():.2%}")
print(f"Challenger 평균 Recall: {challenger_recalls.mean():.2%}")
print(f"차이: {(challenger_recalls.mean() - champion_recalls.mean()):.2%}")
print()

# t-검정 수행
t_stat, p_value = stats.ttest_ind(challenger_recalls, champion_recalls)

print("t-검정 결과")
print(f"t-statistic: {t_stat:.4f}")
print(f"p-value: {p_value:.4f}")
print()

# 결과 해석
if p_value < 0.05:
    print("통계적으로 유의미한 차이가 있습니다 (p < 0.05)")
    print("   -> Challenger가 Champion보다 성능이 좋다고 결론 가능")
else:
    print("통계적으로 유의미한 차이가 없습니다 (p >= 0.05)")
    print("   -> 차이가 우연일 수 있음, 더 많은 데이터 필요")

In [ ]:
# 실습: t-검정 수행

# 새로운 데이터: 14일간 측정한 Precision
champion_precision = np.array([0.72, 0.73, 0.71, 0.72, 0.74, 0.73, 0.72,
                               0.71, 0.73, 0.72, 0.74, 0.73, 0.72, 0.71])
challenger_precision = np.array([0.75, 0.76, 0.74, 0.75, 0.77, 0.76, 0.75,
                                  0.74, 0.76, 0.75, 0.77, 0.76, 0.75, 0.74])

# 1. 평균 계산
champion_mean = champion_precision.mean()
challenger_mean = challenger_precision.mean()

# 2. t-검정 수행
t_stat, p_value = stats.ttest_ind(challenger_precision, champion_precision)

# 3. 결과 출력
print(f"Champion 평균 Precision: {champion_mean:.2%}")
print(f"Challenger 평균 Precision: {challenger_mean:.2%}")
print(f"개선: {challenger_mean - champion_mean:.2%}")
print(f"\np-value: {p_value:.6f}")

# 4. 판단
is_significant = p_value < 0.05

# 체크포인트
assert is_significant == True, "p-value가 0.05보다 작아야 합니다"
print(f"\n체크포인트 2 통과! p={p_value:.6f} < 0.05 -> 통계적으로 유의미")

### 면접에서 p-value 설명하기

```
면접관: "p-value < 0.05가 의미하는 것은?"

모범 답변:
"p-value는 귀무가설(두 모델에 차이가 없다)이 참일 때,
현재 관측한 결과가 나올 확률입니다.

p < 0.05는 이 확률이 5% 미만이라는 뜻으로,
'차이가 없다'는 가정 하에 이런 결과가 나올 확률이 매우 낮다는 의미입니다.

따라서 귀무가설을 기각하고,
'두 모델에 통계적으로 유의미한 차이가 있다'고 결론 내립니다.
현업에서는 5% 유의수준을 기준으로 사용합니다."
```

---
## 4. Champion 승격 조건

### Challenger -> Champion 승격 기준

```
승격 조건 4가지 (모두 충족해야 함)

1. 성능 향상 >= 1%p
   - 의미 있는 개선인지 (노이즈 vs 실제 개선)

2. 통계적 유의성 (p < 0.05)
   - 우연이 아닌지 검증

3. 비용 개선 (또는 동등)
   - 추론 비용, 인프라 비용

4. 최소 관측 기간 >= 2주
   - 충분한 데이터로 판단
   - 주말/평일 패턴, 월초/월말 패턴 반영
```

### 왜 4가지 모두 필요한가?

| 조건 | 빠지면 발생하는 문제 |
|------|---------------------|
| 성능 향상 | 0.1%p 개선에 교체 비용 들임 |
| 통계적 유의성 | 우연한 차이로 잘못된 결정 |
| 비용 개선 | 성능은 좋지만 비용 10배 |
| 관측 기간 | 3일 데이터로 성급한 결정 |

In [ ]:
# Champion 승격 판단 함수

def should_promote_challenger(
    champion_metrics: list,
    challenger_metrics: list,
    champion_cost: float,
    challenger_cost: float,
    observation_days: int,
    min_improvement: float = 0.01,  # 1%p
    p_threshold: float = 0.05,
    min_days: int = 14
):
    """Challenger를 Champion으로 승격할지 판단
    
    Args:
        champion_metrics: Champion 일별 성능 (예: Recall)
        challenger_metrics: Challenger 일별 성능
        champion_cost: Champion 일 운영 비용
        challenger_cost: Challenger 일 운영 비용
        observation_days: 관측 기간 (일)
        
    Returns:
        (승격 여부, 조건별 결과, 상세 정보)
    """
    
    # 1. 평균 성능 비교
    champion_mean = np.mean(champion_metrics)
    challenger_mean = np.mean(challenger_metrics)
    improvement = challenger_mean - champion_mean
    
    # 2. 통계적 유의성
    _, p_value = stats.ttest_ind(challenger_metrics, champion_metrics)
    
    # 3. 승격 조건 체크
    conditions = {
        f"성능 향상 >= {min_improvement:.0%}": improvement >= min_improvement,
        f"p-value < {p_threshold}": p_value < p_threshold,
        "비용 개선": challenger_cost <= champion_cost,
        f"관측 기간 >= {min_days}일": observation_days >= min_days,
    }
    
    # 4. 상세 정보
    details = {
        "champion_mean": champion_mean,
        "challenger_mean": challenger_mean,
        "improvement": improvement,
        "p_value": p_value,
        "cost_saving": champion_cost - challenger_cost,
        "observation_days": observation_days,
    }
    
    all_passed = all(conditions.values())
    
    return all_passed, conditions, details

print("Champion 승격 판단 함수 정의 완료!")

In [ ]:
# 예제: 승격 판단 테스트

# 14일간 Recall 데이터
champion_recall = np.array([0.85, 0.84, 0.86, 0.85, 0.84, 0.85, 0.86,
                            0.85, 0.84, 0.86, 0.85, 0.84, 0.85, 0.86])
challenger_recall = np.array([0.87, 0.88, 0.87, 0.86, 0.88, 0.87, 0.88,
                              0.87, 0.88, 0.87, 0.86, 0.88, 0.87, 0.88])

# 승격 판단
should_promote, conditions, details = should_promote_challenger(
    champion_metrics=champion_recall,
    challenger_metrics=challenger_recall,
    champion_cost=100.0,    # $100/일
    challenger_cost=95.0,   # $95/일 (더 효율적)
    observation_days=14
)

print("승격 조건 검사 결과")
print("=" * 50)
for condition, passed in conditions.items():
    status = "Pass" if passed else "Fail"
    print(f"[{status}] {condition}")

print("\n상세 정보")
print(f"   Champion 평균: {details['champion_mean']:.2%}")
print(f"   Challenger 평균: {details['challenger_mean']:.2%}")
print(f"   개선: {details['improvement']:.2%}")
print(f"   p-value: {details['p_value']:.6f}")
print(f"   비용 절감: ${details['cost_saving']:.2f}/일")

print("\n" + "=" * 50)
if should_promote:
    print("결론: Challenger를 Champion으로 승격!")
else:
    print("결론: 승격 보류, 조건 미충족")

In [ ]:
# 실습: 승격 여부 판단

# 시나리오: 새 모델 테스트 결과
scenario = {
    "champion_recall": np.array([0.80, 0.81, 0.79, 0.80, 0.82, 0.80, 0.81]),
    "challenger_recall": np.array([0.82, 0.83, 0.81, 0.82, 0.84, 0.82, 0.83]),
    "champion_cost": 120.0,
    "challenger_cost": 150.0,  # 더 비쌈!
    "observation_days": 7,     # 7일만 관측
}

should_promote, conditions, details = should_promote_challenger(
    champion_metrics=scenario["champion_recall"],
    challenger_metrics=scenario["challenger_recall"],
    champion_cost=scenario["champion_cost"],
    challenger_cost=scenario["challenger_cost"],
    observation_days=scenario["observation_days"]
)

print("승격 조건 검사 결과")
print("=" * 50)
failed_conditions = []
for condition, passed in conditions.items():
    status = "Pass" if passed else "Fail"
    print(f"[{status}] {condition}")
    if not passed:
        failed_conditions.append(condition)

print(f"\n실패한 조건: {failed_conditions}")

# 체크포인트
assert should_promote == False, "승격하면 안 됩니다!"
assert len(failed_conditions) == 2, "실패 조건은 2개여야 합니다"
print("\n체크포인트 3 통과!")
print("   -> 비용 증가 + 관측 기간 부족으로 승격 보류!")

---
## 5. 최종 요약

In [ ]:
print("="*60)
print("  2-S5 완료: A/B 테스트 기초")
print("="*60)
print()
print("배운 것:")
print()
print("1. A/B 테스트 vs Shadow 모드")
print("   - FDS는 Shadow 모드 사용 (리스크 제로)")
print("   - Challenger는 로깅만, 응답은 Champion만")
print()
print("2. 통계적 유의성 (면접 필수!)")
print("   - p < 0.05: 통계적으로 유의미한 차이")
print("   - t-검정으로 두 모델 비교")
print()
print("3. Champion 승격 조건 4가지")
print("   - 성능 향상 >= 1%p")
print("   - p-value < 0.05")
print("   - 비용 개선")
print("   - 관측 기간 >= 2주")
print()
print("="*60)
print("Phase 2 학습 완료! 구현 노트북으로!")
print("="*60)

### 학습 체크리스트

| 항목 | 이해도 |
|------|--------|
| Shadow 모드 (FDS 특수성) | |
| **p-value < 0.05 (면접!)** | |
| Champion 승격 조건 4가지 | |

### 면접 예상 질문

**1. "FDS에서 Shadow 모드 A/B를 사용하는 이유는?"**

답변:
- FDS는 실제 거래에 영향을 미치므로 일반 A/B 테스트의 리스크가 큼
- 새 모델이 사기를 놓치면 **금전 피해 발생**
- Shadow 모드는 Challenger 모델이 '그림자'처럼 **예측만 로깅**하고, 실제 응답은 검증된 Champion 모델만 함
- 실제 거래에 영향 없이 안전하게 새 모델 성능 평가 가능

---

**2. "p-value < 0.05가 의미하는 것은?"**

답변:
- p-value는 **귀무가설(두 모델에 차이가 없다)이 참일 때, 현재 관측한 결과가 나올 확률**
- p < 0.05는 이 확률이 5% 미만이라는 뜻
- '차이가 없다'는 가정 하에 이런 결과가 나올 확률이 매우 낮음
- 따라서 **귀무가설을 기각**하고, '두 모델에 통계적으로 유의미한 차이가 있다'고 결론
- 현업에서는 **5% 유의수준**을 표준으로 사용

---

**3. "Champion 승격 조건은?"**

답변:
1. **성능 향상 >= 1%p**: 의미 있는 개선인지
2. **통계적 유의성 (p < 0.05)**: 우연이 아닌지
3. **비용 개선**: 추론 비용, 인프라 비용
4. **관측 기간 >= 2주**: 충분한 데이터로 판단